# 💡 Homography and Perspective Transform in OpenCV

Welcome to a **comprehensive guide** on Homography and Perspective Transformation in OpenCV.
This notebook is structured to take you from a beginner to an advanced OpenCV user.

### What you will learn:
- How Homography works mathematically
- How to compute homography matrices using OpenCV
- How to perform perspective warping on images
- How to handle noisy or imperfect correspondences using RANSAC
- Real-world use cases like document scanning, AR, and image stitching

Homography allows you to **map points from one plane to another**, which is more general than affine transformations. It preserves lines but can handle perspective distortions.

## 🔹 Mathematical Intuition

The homography matrix `H` is a 3x3 matrix that transforms a point `(x, y)` in the source image to `(x', y')` in the destination image:

$$ \begin{bmatrix} x' \\ y' \\ w' \end{bmatrix} = H \begin{bmatrix} x \\ y \\ 1 \end{bmatrix}, \quad H = \begin{bmatrix} h_{11} & h_{12} & h_{13} \\ h_{21} & h_{22} & h_{23} \\ h_{31} & h_{32} & 1 \end{bmatrix} $$

After dividing by `w'`:
$$ x' = x'/w', \quad y' = y'/w' $$

- Homography is **projective**: it can handle perspective effects.
- Affine is a special case (last row `[0, 0, 1]`).
- We need at least **4 point correspondences** to solve for H.

In [ ]:
# Imports and Setup
import cv2
import numpy as np
import os
from tools.tools import LearnTools

learn_tools = LearnTools()
%matplotlib inline

In [ ]:
# Load image
# img_url = "https://i.ibb.co/5x276TvQ/1.jpg"
# img_url = "https://i.ibb.co/QjkCQ6Vm/2.jpg"
img_url = "https://i.ibb.co/NyT8LB5/test.jpg"


if os.path.exists('testImage.jpg'):
    image = cv2.imread('testImage.jpg')
else:
    pil_image = await learn_tools.get_image(img_url=img_url, padding=0)
    pil_image.save('testImage.jpg', 'JPEG')
    image = learn_tools.pil_to_cv2(pil_image)

learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image', 'image': image}
        ]
    )

## 🔹 Define Corresponding Points

To compute homography, select **at least 4 points** in the source image and their corresponding positions in the destination plane.

Tips:
- Choose points that are spread across the image.
- Avoid points that are too close to each other.
- For interactive selection, OpenCV's `cv2.setMouseCallback` can be used.

In [ ]:
rows, cols = image.shape[:2]

# Source points in clockwise order (top-left, top-right, bottom-right, bottom-left)
pts_src = np.float32([[50, 50], [200, 50], [200, 200], [50, 200]])

# Destination points simulating a perspective transform
pts_dst = np.float32([[10, 100], [220, 50], [210, 210], [30, 220]])

# Visualize source points
img_vis = image.copy()
for pt in pts_src:
    cv2.circle(img_vis, tuple(pt.astype(int)), 6, (0,0,255), -1)
learn_tools.show_multiple_images([
    {'title': 'Original Image with Source Points', 'image': img_vis}
])

## 🔹 Compute Homography

Use `cv2.findHomography` to compute the 3x3 matrix `H`:

- The function can also use **RANSAC** to handle noisy points.
- `status` shows which points were considered inliers.

In [ ]:
H, status = cv2.findHomography(pts_src, pts_dst)
print("Homography Matrix:\n", H)

## 🔹 Apply Perspective Transform

Use `cv2.warpPerspective` to apply the homography matrix to the entire image:

- This maps the original image onto the new plane defined by `pts_dst`.
- The size `(cols, rows)` ensures the output image has the same dimensions.

In [ ]:
dst = cv2.warpPerspective(image, H, (cols, rows))
learn_tools.show_multiple_images([
    {'title': 'Original Image', 'image': image},
    {'title': 'Warped Image', 'image': dst}
])

## 🔹 Robust Homography with RANSAC

In real-world scenarios, point correspondences may have noise. **RANSAC** helps compute a robust homography:

- It iteratively selects random subsets of points.
- Finds the homography that maximizes the number of inliers.
- Minimizes the effect of outliers.

In [ ]:
H_ransac, status_ransac = cv2.findHomography(pts_src, pts_dst, cv2.RANSAC)
dst_ransac = cv2.warpPerspective(image, H_ransac, (cols, rows))
learn_tools.show_multiple_images([
    {'title': 'Warped Image with RANSAC', 'image': dst_ransac}
])

## 🌍 Real-World Use Cases
- **Document Scanning**: Correct skewed documents automatically.
- **Image Stitching**: Align overlapping photos to create panoramas.
- **Augmented Reality**: Map virtual objects onto planar surfaces.
- **Object Tracking**: Track planar objects under perspective changes.
- **Video Stabilization**: Align frames to reduce camera shake.

## ✅ Best Practices and Tips
- Use **at least 4 widely spread points** to ensure a stable solution.
- Always visualize points before and after transformation.
- Use `cv2.RANSAC` for robustness against incorrect correspondences.
- For **image stitching**, blend overlapping areas to remove seams.
- Combine homography with other transformations (affine, scaling) for complex pipelines.